# ECON 490: Generating Variables (5)

## Prerequisites
---
1. Import datasets in csv and dta format.
2. Save files.

## Learning Outcomes
---
In this module, you will learn to:

1. Generate dummy (or indicator) variables using `ifelse` and `case_when`.
2. Create new variables using `mutate`.
3. Rename variables using `rename`.

## 5.1 Review of the Data Opening Procedure
We'll continue working with the fake data data set introduced in the previous lecture. Recall that this data set is simulating information of workers in the years 1982-2012 in a fake country where a training program was introduced in 2003 to boost their earnings.  

Last lecture we introduced the process of loading our "fake_data" data set into R.
1. Import the relevant package (Haven) which gives us access to commands for loading the data. Also import the tidyverse package in order to clean our data.
2. Use the `read_csv` or `read_dta` functions to load our data set. 
3. Clean our data by factorizing all important variables.

Let's run through this procedure quickly so that we are all ready to do our analysis. 

In [1]:
library(haven)
library(tidyverse)

Warning message:
“package ‘haven’ was built under R version 4.1.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tibble’ was built under R version 4.1.3”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
data <- read_csv("../econ490-stata/fake_data.csv")
data <- as_factor(data)

##  5.2 Generating Dummy Variables
The most common type of variable we will create during data cleaning and analysis is the dummy variable. Dummy variables are variables that can only take on two values: 0 and 1. It is useful to think of a dummy variable as being the answer to a question that can be answered "yes" or "no". With a dummy variable the answer yes is coded as "1" and no is coded as "0".

Examples of question that are used to create dummy variables can include:

1. Is the person female? Females are coded "1" and males are coded "0".
2. Does the person have a university degree? People with a university degree are coded "1" and everyone else is coded "0".
3. Is the person married? Married people are coded "1" and everyone else is coded "0".
4. Is the person a millennial? People born between 1980 and 1996 are coded "1" and those born in other years are coded "0".

As you have probably already figured out, dummy variables are used primarily for data that is qualitative and cannot be ranked in any way. For example, being married is qualitative and "married" is neither higher nor lower than "single".  However, dummy variables sometimes also refer to variables that are qualitative and ranked, such as level of education, and sometimes for variables that are quantitative, such as age groupings. 

It is important to remember that dummy variables must always be used when we want to include categorical (qualitative) variables in our analysis. These are variables such as sex, gender, race, marital status, religiosity, immigration status etc. Without creating dummy variables for these demographics, analysis of the results from data analysis, regression, and other research will not be meaningful, as we are working with variables which have been numerically scaled in an arbitrary way. This is especially true for interpreting the coefficients outputted from a regression.

#### 5.2.1 Creating dummy variables using `ifelse`

Let's do an example where we create a dummy variable that indicates if the observation identified as female. We are going to use the command `ifelse` which generates a completely new variable based on certain conditions. 

In [ ]:
data$female = ifelse(data$sex == "F", 1, 0)

What R interprets here is that if the condition `sex == "F" ` holds, then our dummy will take the value of 1, else it will take the value of zero. This is where the `ifelse` functional component comes in. Depending on what you're doing, you may want it to be the case that when `sex` is missing, our dummy is zero. We can first check if we have any missing observations for a given variable by using the `is.na` function nested within the `any` function. If there are any missing values for the `sex` variable in this dataset, the code below will return TRUE.

In [ ]:
sum(is.na(data$sex))

If we want to account for missing values and ensure that they are denoted as 0 for the dummy `female`, we can again invoke the `is.na` function as an additional condition in our function.

In [ ]:
data$female = ifelse(data$sex == "F" & !is.na(data$sex), 1, 0)

The above condition within our function says that `female` == 1 only when `sex` == "F" and `sex` is not marked as NA (since !is.na must be TRUE).

#### 5.2.2 Creating a series of dummy variables using `ifelse`

We now know how to create dummy variables with `ifelse`. However, we may also want to create dummy variables corresponding to a whole set of categories for a given variable - for example, one for each region identified in the data set. To do this, we can just meticulously craft a dummy for each category, such as `reg1`, `reg2`, `reg3`, and `reg4`. We must leave out one region to serve as our base group, being region 5, in order to avoid the dummy variable trap.

In [ ]:
data$reg1 = ifelse(data$region == 1 & !is.na(data$region), 1, 0)
data$reg2 = ifelse(data$region == 2 & !is.na(data$region), 1, 0)
data$reg3 = ifelse(data$region == 3 & !is.na(data$region), 1, 0)
data$reg4 = ifelse(data$region == 4 & !is.na(data$region), 1, 0)

This command generated five new dummy variables, one for each category for region. We asked Stata to call those variables "reg" and so these new dummy variables are called reg1, reg2, reg3, reg4. This is quite cumbersome. There are packages out there which help to expedite this process. Luckily, if we are running a regression on a qualitative variable such as `region`, R will generate the necessary dummy variables for us automatically.

#### 5.2.3 Creating dummy variables using `case_when`

We can also use more complex functions to create dummy variables. The most important one we will mention is the `case_when` function. This function creates different values for an input based on specified cases. Specifically, it consists of a series of lines, and each line gives a (i) case and (ii) value for that case. This function is nearly always used to operate on strings, or variables which do not have numerical significance in terms of how they are coded. Otherwise, we could use simple operators such as <, >, and = to classify values of these variables and then invoke the `ifelse` function as we did above. Unfortunately, we don't have any variables in our "fake_data" data set which call for this and we don't have an example fit for this function. However, for some documentation of the `case_when` function, run the code cell below!

In [ ]:
?case_when

## 5.3 Generating Variables Based on Expressions
Sometimes we want to generate variables after some transformations (e.g. squaring, taking logs, combining different variables). We can do that by simply writing the expression as an argument to the function `mutate`. This function manipulates our dataframe by supply to it a new column based on the function we input. For example, let's create a variable called `log_earnings` which is the log of earnings.

In [ ]:
data <- data %>% mutate(log_earnings = log(earnings))

summary(data$log_earnings)

Let's try a second example. Let's create a new variable that is the number of years since the year the individual started working. 

In [ ]:
data <- data %>% mutate(experience_proxy = year - start_year)

summary(data$experience_proxy)

The `mutate` function allows us to easily add new variables to our dataframe. If we wanted to instead replace a given variable with a new feature, say add one default year to all experience_proxy observations, we can simply redefine it directly in our dataframe.

In [ ]:
data$experience_proxy <- data$experience_proxy + 1

## 5.4 Following Good Naming Conventions
Choosing good names for your variables is more important, and harder, than you might think! Sometimes, the variables in a dataset have unrecognizable names, which may be confusing when conducting research. In these cases, it is a good idea to change them immediately. In your research, you will also be creating your own variables (like dummy variables) for qualitative measures and will want to be careful about giving them good names. This is especially important for generating tables, since you will want your tables to be easily legible in your paper.


We can rename variables with the `rename` function found inside the `dplyr` package (which we can access via having loaded in R's tidyverse). Let' try to rename one of those dummy variables we created above. Maybe we know that if region = 3 then the region is in the west.

In [ ]:
rename(data, west = reg3)

Don’t think that you need to include every piece of information in your variable names. Most of the important information pertaining to a variable is included in its label (more on that in a moment). Avoid variable names that include unnecessary pieces of information and can only be interpreted by you. 

## 5.5 Wrap Up
When we are doing your own research, we *always* have to spend some time working with the data before beginning the analysis. In this module we have learned some important tools for manipulating data to get it ready for that analysis. Like everything else that you do in R, emphasis should be on readibility and reproducibility in your code. This is pivotal for you and your audience to understand your research. In the next module, we will explore how to create new variables for group level analysis, among other things.